In [1]:
import tarfile
import shutil
import os
import json

In [3]:
old_train_path = "/kaggle/input/kitti-full-dataset/KITTI Dataset/KITTI Dataset/train"
new_images_path = "dataset/images"
new_labels_path = "dataset/labels"

os.makedirs(os.path.join(new_images_path,"train"),exist_ok=True)
os.makedirs(os.path.join(new_images_path,"val"),exist_ok=True)
os.makedirs(os.path.join(new_labels_path,"train"),exist_ok=True)
os.makedirs(os.path.join(new_labels_path,"val"),exist_ok=True)

img_list = sorted(os.listdir(os.path.join(old_train_path,"img")))
total_images = len(img_list)
threshold = int(total_images*0.8)

for index,img in enumerate(img_list):
    old_path = os.path.join(old_train_path,"img",img)
    if(index<=threshold):
      new_path = os.path.join(new_images_path,"train",img)
    else:
      new_path = os.path.join(new_images_path,"val",img)
    shutil.copy(old_path,new_path)

print("Images copied")

Images copied


In [4]:
class_mapping = {
    "car": 0,
    "van": 1,
    "truck": 2,
    "pedestrian": 3,
    "person sitting": 4,
    "cyclist": 5,
    "tram": 6,
    "misc": 7,
    "dont care": -1  # Skip "dont care" objects
}

ann_list = sorted(os.listdir(os.path.join(old_train_path,"ann")))

for index,annotation in enumerate(ann_list):
  ann_path = os.path.join(old_train_path,"ann",annotation)
  with open(ann_path) as f:
    data = json.load(f)
    img_height = data["size"]["height"]
    img_width = data["size"]["width"]
    yolo_notations = []
    for obj in data["objects"]:
      class_name = obj["classTitle"]
      class_id = class_mapping.get(class_name, -1)
      if class_id == -1:
        continue
      x_min,y_min = obj["points"]["exterior"][0]
      x_max,y_max = obj["points"]["exterior"][1]
      x_center = ((x_min+x_max)/2)/img_width
      y_center = ((y_min+y_max)/2)/img_height
      width = (x_max-x_min)/img_width
      height = (y_max-y_min)/img_height
      yolo_notations.append(f"{class_id} {x_center} {y_center} {width} {height}")
    new_ann_file_path = annotation.split(".")[0]+".txt"
    if(index<=threshold):
      new_ann_file_path = os.path.join(new_labels_path,"train",new_ann_file_path)
    else:
      new_ann_file_path = os.path.join(new_labels_path,"val",new_ann_file_path)

    with open(os.path.join(new_ann_file_path),"w") as f:
        f.write("\n".join(yolo_notations))
print("Annotations copied")

Annotations copied


In [5]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.2/881.2 kB 15.6 MB/s eta 0:00:00a 0:00:01


In [ ]:
shutil.rmtree("/kaggle/working/runs")

In [8]:
from ultralytics import YOLO

model = YOLO('yolov8l.pt')  

# Train the model
model.train(
    data='/kaggle/input/kitti-full-dataset/kitti_kaggle2.yaml',   # Path to the data.yaml file
    epochs=100,                 # Number of training epochs
    imgsz=640,                  # Image size
    batch=32,                   # Batch size
    name='yolov8l_custom',       # Name for the training run
    project='runs/train',        # Directory to save training results
    device="0,1",                 # Use GPU 0 and GPU 1
    # optimizer="Adam"            # Use the Adam optimizer
)

Ultralytics 8.3.28 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/kaggle/input/kitti-full-dataset/kitti_kaggle2.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0,1, workers=8, project=runs/train, name=yolov8l_custom2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, s

train: Scanning /kaggle/working/dataset/labels/train.cache... 5985 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5985/5985 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/dataset/labels/val.cache... 1496 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1496/1496 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
Plotting labels to runs/train/yolov8l_custom2/labels.jpg... 


/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/train/yolov8l_custom2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1496       7982      0.953      0.881      0.936      0.766

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      10.6G     0.4082     0.2284     0.8105          4        640: 100%|██████████| 188/188 [02:40<00:00,  1.17it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:14<00:00,  3.20it/s]


                   all       1496       7982      0.955       0.88      0.935      0.766

100 epochs completed in 4.935 hours.
Optimizer stripped from runs/train/yolov8l_custom2/weights/last.pt, 87.6MB
Optimizer stripped from runs/train/yolov8l_custom2/weights/best.pt, 87.6MB

Validating runs/train/yolov8l_custom2/weights/best.pt...
Ultralytics 8.3.28 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
Model summary (fused): 268 layers, 43,612,776 parameters, 0 gradients, 164.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:15<00:00,  2.99it/s]


                   all       1496       7982      0.945      0.879      0.937      0.768
                   car       1337       5687      0.966      0.949      0.984       0.88
                   van        420        552      0.944      0.949      0.969      0.856
                 truck        207        219      0.964      0.991      0.986      0.901
            pedestrian        348        872      0.952      0.754      0.891       0.56
        person sitting         17         49       0.86      0.673      0.809      0.607
               cyclist        209        292      0.947      0.884      0.931      0.719
                  tram         71        106      0.956      0.934      0.976      0.828
                  misc        164        205      0.973      0.898      0.953      0.791
Speed: 0.1ms preprocess, 5.7ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/train/yolov8l_custom2


In [9]:
metrics = model.val(data='/kaggle/input/kitti-full-dataset/kitti_kaggle2.yaml')
print(metrics)

Ultralytics 8.3.28 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
Model summary (fused): 268 layers, 43,612,776 parameters, 0 gradients, 164.8 GFLOPs


val: Scanning /kaggle/working/dataset/labels/val.cache... 1496 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1496/1496 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:27<00:00,  1.73it/s]


                   all       1496       7982      0.923      0.901       0.94      0.768
                   car       1337       5687      0.946      0.961      0.984      0.881
                   van        420        552      0.918      0.958      0.969       0.86
                 truck        207        219      0.952      0.995      0.986      0.903
            pedestrian        348        872      0.921      0.811       0.89      0.562
        person sitting         17         49      0.837      0.731      0.828      0.607
               cyclist        209        292      0.926      0.895      0.931       0.72
                  tram         71        106      0.944       0.95      0.976      0.825
                  misc        164        205      0.942      0.907      0.953       0.79
Speed: 0.1ms preprocess, 13.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/train/yolov8l_custom22
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_

In [12]:
results = model.predict(source='/kaggle/working/dataset/images/val/006320.png',device="cpu" ,conf=0.5)


Ultralytics 8.3.28 🚀 Python-3.10.14 torch-2.4.0 CPU (Intel Xeon 2.00GHz)
YOLOv8l summary (fused): 268 layers, 43,668,288 parameters, 0 gradients, 165.2 GFLOPs
image 1/1 /kaggle/working/dataset/images/val/006320.png: 224x640 4 cars, 4 traffic lights, 450.6ms
Speed: 1.7ms preprocess, 450.6ms inference, 10.2ms postprocess per image at shape (1, 3, 224, 640)


In [16]:
from IPython.display import FileLink
!zip -r /kaggle/working/yolo8l.zip /kaggle/working/runs/train/yolov8l_custom2
FileLink('/kaggle/working/yolo8l.zip')

  adding: kaggle/working/runs/train/yolov8l_custom2/ (stored 0%)
  adding: kaggle/working/runs/train/yolov8l_custom2/val_batch0_labels.jpg (deflated 10%)
  adding: kaggle/working/runs/train/yolov8l_custom2/train_batch2.jpg (deflated 10%)
  adding: kaggle/working/runs/train/yolov8l_custom2/PR_curve.png (deflated 11%)
  adding: kaggle/working/runs/train/yolov8l_custom2/val_batch2_labels.jpg (deflated 10%)
  adding: kaggle/working/runs/train/yolov8l_custom2/train_batch16922.jpg (deflated 21%)
  adding: kaggle/working/runs/train/yolov8l_custom2/results.png (deflated 9%)
  adding: kaggle/working/runs/train/yolov8l_custom2/F1_curve.png (deflated 7%)
  adding: kaggle/working/runs/train/yolov8l_custom2/labels_correlogram.jpg (deflated 32%)
  adding: kaggle/working/runs/train/yolov8l_custom2/P_curve.png (deflated 10%)
  adding: kaggle/working/runs/train/yolov8l_custom2/weights/ (stored 0%)
  adding: kaggle/working/runs/train/yolov8l_custom2/weights/last.pt (deflated 8%)
  adding: kaggle/working

/kaggle/working/yolo8l.zip